In [71]:
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
import cv2
from tensorflow.keras.models import load_model
from mtcnn import MTCNN


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mtcnn import MTCNN
from keras.models import load_model

# Load model yang sudah dilatih
model = load_model("model/5kelas.keras")

image_path = 'test-img/image7.png'

# Inisialisasi detektor wajah MTCNN
detector = MTCNN()

# Mapping indeks ke nama kelas
class_names = ['angry', 'happy', 'neutral', 'sad', 'surprised']

def detect_and_crop_faces(image):
    """
    Fungsi untuk mendeteksi wajah dan meng-crop wajah yang terdeteksi.
    :param image: Input gambar (numpy array)
    :return: List gambar wajah yang ter-crop, atau None jika tidak ada wajah yang terdeteksi
    """
    faces = detector.detect_faces(image)  # Deteksi wajah dalam gambar
    cropped_faces = []
    
    for face in faces:
        x, y, w, h = face['box']  # Ambil koordinat wajah
        cropped_face = image[y:y+h, x:x+w]  # Crop wajah dari gambar
        cropped_faces.append(cropped_face)
    
    return cropped_faces

def predict_image(image_array):
    """
    Fungsi untuk memprediksi emosi dari gambar input.
    :param image_array: Array gambar wajah yang sudah di-crop
    :return: Nama kelas prediksi
    """
    image_resized = cv2.resize(image_array, (100, 100))  # Resize ke ukuran input model
    image_resized = np.expand_dims(image_resized, axis=0)  # Tambahkan dimensi batch
    image_resized = image_resized / 255.0  # Normalisasi gambar
    predictions = model.predict(image_resized, verbose=0)  # Prediksi
    predicted_class = np.argmax(predictions)  # Ambil kelas dengan probabilitas tertinggi
    return class_names[predicted_class]

# Baca gambar dari file
image = cv2.imread(image_path)

# Cek apakah gambar berhasil dibaca
if image is None:
    print(f"Error: Unable to load image at {image_path}")
else:
    # Convert BGR (OpenCV) ke RGB (matplotlib)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Deteksi dan crop wajah
    cropped_faces = detect_and_crop_faces(image_rgb)

    if cropped_faces:
        plt.figure(figsize=(500, 500))
        
        # Menampilkan gambar asli
        plt.subplot(1, len(cropped_faces) + 1, 1)
        plt.imshow(image_rgb)
        plt.title("Original Image")
        plt.axis('off')

        # Menampilkan wajah-wajah yang terdeteksi beserta prediksi emosi
        for i, cropped_face in enumerate(cropped_faces):
            predicted_emotion = predict_image(cropped_face)
            plt.subplot(1, len(cropped_faces) + 1, i + 2)
            plt.imshow(cropped_face)
            plt.title(f"Face {i+1}\nEmotion: {predicted_emotion}")
            plt.axis('off')

        plt.show()
    else:
        print("Tidak ada wajah yang terdeteksi!")
